# Lumira Tiny LLM - Training Notebook

Google Colab用のLumira Transformer訓練ノートブック

## セットアップ
1. ランタイム → ランタイムのタイプを変更 → GPU (T4)
2. 上から順にセルを実行

In [ ]:
# Google Driveをマウント（チェックポイント保存用）
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# リポジトリをクローン（クリーンアップしてから）
%cd /content
!rm -rf Lumira
!git clone https://github.com/iwamaki/Lumira.git
%cd /content/Lumira

# 確認
!pwd
!ls

In [ ]:
# 依存関係をインストール
!pip install -q torch sentencepiece tokenizers tqdm pandas gradio

In [ ]:
# GPU確認
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Step 1: データ生成

語彙を拡張し、訓練データを生成します。

In [ ]:
# データ生成
!python scripts/generate_data.py \
    --vocab-size 500 \
    --data-size 100000

In [ ]:
# 生成されたデータを確認
!wc -l data/processed/train.jsonl data/processed/val.jsonl
!head -5 data/processed/train.jsonl

## Step 2: トークナイザー訓練

In [ ]:
# トークナイザーを訓練
!python scripts/train_tokenizer.py \
    --vocab-size 8000

## Step 3: モデル訓練

In [ ]:
# 訓練設定
import sys
sys.path.insert(0, '.')

from src.model import ModelConfig, TINY_CONFIG

# モデルサイズ確認
config = TINY_CONFIG
print(f"Model config: {config}")
print(f"Estimated parameters: {config.estimate_params():,}")

In [ ]:
# 訓練開始
from src.training import Trainer, TrainingConfig

config = TrainingConfig(
    train_data="data/processed/train.jsonl",
    val_data="data/processed/val.jsonl",
    tokenizer_model="data/vocab/lumira.model",
    output_dir="/content/drive/MyDrive/lumira_checkpoints",  # Google Driveに保存
    model_config="tiny",
    epochs=10,
    batch_size=24,  # T4 GPU向け
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    use_amp=True,
    save_every=500,
    eval_every=250,
)

trainer = Trainer(config)
trainer.train()

## Step 4: 訓練再開（セッション切断後）

In [ ]:
# 訓練を再開する場合
from src.training import Trainer, TrainingConfig

config = TrainingConfig(
    train_data="data/processed/train.jsonl",
    val_data="data/processed/val.jsonl",
    tokenizer_model="data/vocab/lumira.model",
    output_dir="/content/drive/MyDrive/lumira_checkpoints",
    model_config="tiny",
    epochs=10,
    batch_size=24,
    gradient_accumulation_steps=4,
    use_amp=True,
    resume_from="/content/drive/MyDrive/lumira_checkpoints/step_1000.pt",  # チェックポイントを指定
)

trainer = Trainer(config)
trainer.train()

## Step 5: 推論テスト

In [ ]:
from src.inference import Translator
from src.model import TINY_CONFIG

translator = Translator(
    model_path="/content/drive/MyDrive/lumira_checkpoints/best.pt",
    tokenizer_path="data/vocab/lumira.model",
    model_config=TINY_CONFIG,  # 訓練時と同じ設定を使用
)

# テスト翻訳
test_sentences = [
    "こんにちは",
    "ありがとう",
    "私はあなたを愛しています",
    "今日は美しい日です",
    "私たちは成長している",
]

for sent in test_sentences:
    result = translator.translate(sent)
    print(f"{sent} → {result}")

## Step 6: Gradio デモ

In [ ]:
from src.inference.translate import create_demo_ui

demo = create_demo_ui(translator)
demo.launch(share=True)  # share=Trueで公開URLを生成